In [1]:
import datetime
import podpac
from podpac.datalib import smap_egi
import xarray as xr
import zarr
import numpy as np

C:\Anaconda3\envs\podpac\lib\site-packages\xarray\core\merge.py:17: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  PANDAS_TYPES = (pd.Series, pd.DataFrame, pd.Panel)


In [2]:
bucket = "podpac-internal-test"
source = 's3://%s/%s' % (bucket, 'SPL3SMP_E.002_9km_aid0001.zarr')
source2 = 's3://%s/%s' % (bucket, 'SPL4SMGP.004_9km_aid0001.zarr')

kwargs = {'dims': ['time', 'lat', 'lon'], 'latkey': 'fakedim0', 'lonkey': 'fakedim1',
          'cf_time': True, 'cf_calendar': 'proleptic_gregorian', 'cf_units': 'days since 2000-01-01',
         'crs': 'epsg:6933', 'nan_vals': [-9999]}
smap_zarr = podpac.data.Zarr(source=source, datakey='Soil_Moisture_Retrieval_Data_AM_soil_moisture', file_mode='a', **kwargs)
smap_ds = smap_zarr.dataset

smap_zarr_L4 = podpac.data.Zarr(source=source2, datakey='Geophysical_Data_sm_surface', file_mode='a', **kwargs)
smap_ds_L4 = smap_zarr_L4.dataset

In [53]:
coords = smap_zarr.native_coordinates.drop('time')
time = podpac.crange(smap_zarr.native_coordinates['time'].bounds[-1], str(datetime.date.today()), '1,D', 'time')
coords = podpac.coordinates.merge_dims([podpac.Coordinates([time], crs=coords.crs), coords])
coords

Coordinates (epsg:6933)
	time: UniformCoordinates1d(time): Bounds[2019-06-20T00:00:00.000000000, 2019-11-25T00:00:00.000000000], N[159], ctype['midpoint']
	lat: ArrayCoordinates1d(lat): Bounds[2878072.923230057, 5589497.541484003], N[302], ctype['midpoint']
	lon: ArrayCoordinates1d(lon): Bounds[-12174386.616512204, -6436255.447649201], N[638], ctype['midpoint']

In [4]:
coords_l4 = smap_zarr_L4.native_coordinates.drop('time')
time = podpac.crange(smap_zarr_L4.native_coordinates['time'].bounds[-1], str(datetime.date.today()), '1,D', 'time')
coords_l4 = podpac.coordinates.merge_dims([podpac.Coordinates([time],  crs=coords_l4.crs), coords_l4])
coords_l4

Coordinates (epsg:6933)
	time: UniformCoordinates1d(time): Bounds[2019-11-22T00:00:00.000000000, 2019-11-25T00:00:00.000000000], N[4], ctype['midpoint']
	lat: ArrayCoordinates1d(lat): Bounds[2878072.923230057, 5589497.541484003], N[302], ctype['midpoint']
	lon: ArrayCoordinates1d(lon): Bounds[-12174386.616512204, -6436255.447649201], N[638], ctype['midpoint']

In [ ]:
# Iterate through each date to get and set all the data
# L4 data
smap = smap_egi.SMAP(product='SPL4SMAU')
time_base = xr.coding.times.decode_cf_datetime(0, smap_ds_L4['time'].attrs['units'])

for c in coords_l4.iterchunks((2, ) + coords_l4.shape[1:]): 
    print("Download L4 Data for:", str(c['time']), '...', end='')
    try: 
        l4 = smap.eval(c)
    except ValueError as e:
        print("No Granules available:", e)
        continue
    l4.set(-9999, np.isnan(l4))
    print(" ... Done.")
    new_times = (c.coords['time'].data - time_base).astype('timedelta64[D]')
    for i, nt in enumerate(new_times):
        if np.any((smap_ds_L4['time'][:] - nt).astype(int) >= 0) or np.all(np.isnan(l4[i])):
            print('Time already exists, or is all nan -- skipping.')
            continue
        print("Updating S3 Zarr file for L4 Data.")
        old_time_shape = smap_ds_L4['time'].shape
        old_data_shape = smap_ds_L4['Geophysical_Data_sm_surface'].shape
        try: 
            smap_ds_L4['time'].append(np.atleast_1d(nt))
            smap_ds_L4['Geophysical_Data_sm_surface'].append(l4[i:i+1], axis=0)
        except Exception as e:
            print ('Updating L4 data failed:', e)
            smap_ds_L4['time'].resize(*old_time_shape)
            smap_ds_L4['Geophysical_Data_sm_surface'].resize(*old_data_shape)
    

In [40]:
smap_ds_L4['time'][-5:]

array([7262, 7263, 7264, 7265, 7266])

In [41]:
smap_ds_L4['time'].shape, smap_ds_L4['Geophysical_Data_sm_surface'].shape

((263,), (263, 302, 638))

In [10]:
# smap_ds_L4['time'].resize(*old_time_shape)
# smap_ds_L4['Geophysical_Data_sm_surface'].resize(*old_data_shape)

In [54]:
old_data_shape, old_time_shape

((112, 302, 638), (112,))

In [78]:
for c in coords.iterchunks((2, ) + coords.shape[1:]): 
    print("Download L3 Data for:", str(c['time']), '...', end='')
    try: 
        # AM Data
        print("AM", end=' ')
        smap = smap_egi.SMAP(product='SPL3SMP_E_AM', check_quality_flags=False)
        l3_am = smap.eval(c)
        smap.set_trait('data_key', smap.quality_flag_key)
        print("AM_QF", end=' ')
        l3_am_qf = smap.eval(c)

        # PM Data
        print("PM", end=' ')
        smap = smap_egi.SMAP(product='SPL3SMP_E_PM', check_quality_flags=False)
        l3_pm = smap.eval(c)
        smap.set_trait('data_key', smap.quality_flag_key)
        print("PM_QF", end=' ')
        l3_pm_qf = smap.eval(c)
    except ValueError as e:
        print("No Granules available:", e)
        continue
    l3_am.set(-9999, np.isnan(l3_am))
    l3_am_qf.set(-9999, np.isnan(l3_am_qf))
    l3_pm.set(-9999, np.isnan(l3_pm))
    l3_pm_qf.set(-9999, np.isnan(l3_pm_qf))    
    
    print(" ... Done.")
    new_times = (c.coords['time'].data - time_base).astype('timedelta64[D]')
    for i, nt in enumerate(new_times):
        if np.any((smap_ds['time'][:] - nt).astype(int) >= 0) or np.all(np.isnan(l3_am)) or np.all(np.isnan(l3_pm)):
            print('Time already exists, or is all nan -- skipping.')
            continue

        print("Updating S3 Zarr file for L3 Data.")
        old_time_shape = smap_ds['time'].shape
        old_data_shape = smap_ds['Soil_Moisture_Retrieval_Data_AM_soil_moisture'].shape

#         breaka

        try:
            # Update the zarr file
            ## IMPORTANT ONLY EXECUTE THIS CELL ONCE!!!! ANY ERRORS? DO NOT EXECUTE THE SAME APPEND AGAIN!
            smap_ds['time'].append(np.atleast_1d(nt))
            smap_ds['Soil_Moisture_Retrieval_Data_AM_soil_moisture'].append(l3_am[i:i+1], axis=0)
            smap_ds['Soil_Moisture_Retrieval_Data_AM_retrieval_qual_flag'].append(l3_am_qf[i:i+1], axis=0)
            smap_ds['Soil_Moisture_Retrieval_Data_PM_soil_moisture_pm'].append(l3_pm[i:i+1], axis=0)
            smap_ds['Soil_Moisture_Retrieval_Data_PM_retrieval_qual_flag_pm'].append(l3_pm_qf[i:i+1], axis=0)
        except Exception as e:
            print ('Updating L3 data failed:', e)
            smap_ds['time'].resize(*old_time_shape)
            smap_ds['Soil_Moisture_Retrieval_Data_AM_soil_moisture'].resize(*old_data_shape)
            smap_ds['Soil_Moisture_Retrieval_Data_AM_retrieval_qual_flag'].resize(*old_data_shape)
            smap_ds['Soil_Moisture_Retrieval_Data_PM_soil_moisture_pm'].resize(*old_data_shape)
            smap_ds['Soil_Moisture_Retrieval_Data_PM_retrieval_qual_flag_pm'].resize(*old_data_shape)
    
    
    

Download L3 Data for: UniformCoordinates1d(time): Bounds[2019-06-20T00:00:00.000000000, 2019-06-21T00:00:00.000000000], N[2], ctype['midpoint'] ...AM No Granules available: Failed to download data from EGI Interface. EGI Reponse: <?xml version="1.0" encoding="UTF-8" standalone="yes"?>
<eesi:Exception xsi:schemaLocation="http://eosdis.nasa.gov/esi/rsp/e https://newsroom.gsfc.nasa.gov/esi/8.1/schemas/ESIAgentResponseExternal.xsd" xmlns="" xmlns:iesi="http://eosdis.nasa.gov/esi/rsp/i" xmlns:ssw="http://newsroom.gsfc.nasa.gov/esi/rsp/ssw" xmlns:eesi="http://eosdis.nasa.gov/esi/rsp/e" xmlns:esi="http://eosdis.nasa.gov/esi/rsp" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance">
    <Code>InternalError</Code>
    <Message>No granules returned by CMR: https://cmr.earthdata.nasa.gov/search/granules?provider=NSIDC_ECS&amp;page_num=1&amp;short_name=SPL3SMP_E&amp;version=003&amp;temporal=2019-06-20T00:00:00,2019-06-21T00:00:00&amp;token=C068ED4B-9C70-B9FA-9CF7-64E4A35346B0&amp;page_size=20</Me

AM_QF 

PM PM_QF  ... Done.
Time already exists, or is all nan -- skipping.
Time already exists, or is all nan -- skipping.
Download L3 Data for: UniformCoordinates1d(time): Bounds[2019-07-24T00:00:00.000000000, 2019-07-25T00:00:00.000000000], N[2], ctype['midpoint'] ...AM 

c:\repositories\pipeline\podpac\podpac\core\coordinates\dependent_coordinates.py:496: UserWarning: transformation of coordinate segment lengths not yet implemented
  warnings.warn("transformation of coordinate segment lengths not yet implemented")
c:\repositories\pipeline\podpac\podpac\core\coordinates\dependent_coordinates.py:498: UserWarning: transformation of coordinate segment lengths not yet implemented
  warnings.warn("transformation of coordinate segment lengths not yet implemented")


AM_QF 

...

PM_QF 

c:\repositories\pipeline\podpac\podpac\datalib\smap_egi.py:204: RuntimeWarning: Mean of empty slice
  lon = np.nanmean(lons, axis=0)
c:\repositories\pipeline\podpac\podpac\core\coordinates\array_coordinates1d.py:83: RuntimeWarning: invalid value encountered in less_equal
  if np.any(deltas <= 0):
c:\repositories\pipeline\podpac\podpac\core\coordinates\dependent_coordinates.py:496: UserWarning: transformation of coordinate segment lengths not yet implemented
  warnings.warn("transformation of coordinate segment lengths not yet implemented")
c:\repositories\pipeline\podpac\podpac\core\coordinates\dependent_coordinates.py:498: UserWarning: transformation of coordinate segment lengths not yet implemented
  warnings.warn("transformation of coordinate segment lengths not yet implemented")
c:\repositories\pipeline\podpac\podpac\core\cache\ram_cache_store.py:88: UserWarning: Warning: Process is using more RAM than the specified limit in settings.RAM_CACHE_MAX_BYTES. No longer caching. Consider

 ... Done.
Updating S3 Zarr file for L3 Data.
Updating S3 Zarr file for L3 Data.
Download L3 Data for: UniformCoordinates1d(time): Bounds[2019-11-25T00:00:00.000000000, 2019-11-25T00:00:00.000000000], N[1], ctype['midpoint'] ...AM 

c:\repositories\pipeline\podpac\podpac\core\coordinates\dependent_coordinates.py:496: UserWarning: transformation of coordinate segment lengths not yet implemented
  warnings.warn("transformation of coordinate segment lengths not yet implemented")
c:\repositories\pipeline\podpac\podpac\core\coordinates\dependent_coordinates.py:498: UserWarning: transformation of coordinate segment lengths not yet implemented
  warnings.warn("transformation of coordinate segment lengths not yet implemented")


No Granules available: Failed to download data from EGI Interface. EGI Reponse: <?xml version="1.0" encoding="UTF-8" standalone="yes"?>
<eesi:Exception xsi:schemaLocation="http://eosdis.nasa.gov/esi/rsp/e https://newsroom.gsfc.nasa.gov/esi/8.1/schemas/ESIAgentResponseExternal.xsd" xmlns="" xmlns:iesi="http://eosdis.nasa.gov/esi/rsp/i" xmlns:ssw="http://newsroom.gsfc.nasa.gov/esi/rsp/ssw" xmlns:eesi="http://eosdis.nasa.gov/esi/rsp/e" xmlns:esi="http://eosdis.nasa.gov/esi/rsp" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance">
    <Code>InternalError</Code>
    <Message>No granules returned by CMR: https://cmr.earthdata.nasa.gov/search/granules?provider=NSIDC_ECS&amp;page_num=1&amp;short_name=SPL3SMP_E&amp;version=003&amp;temporal=2019-11-25T00:00:00,2019-11-25T00:00:00&amp;token=C068ED4B-9C70-B9FA-9CF7-64E4A35346B0&amp;page_size=20</Message>
</eesi:Exception>

